<a href="https://colab.research.google.com/github/victormuelacarriles/GoogleColab/blob/main/Probando_Hashcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Probando Hascat desde los servidores de google**
Esta prueba se basa en modificaciones del proyecto Cloudtopolis (https://github.com/JoelGMSec/Cloudtopolis).

En lugar de buscar multiples sesiones, simplemente se quiere ejecutar una sesión para resolver hashes (en concreto de NTML).
Se ha divido en 3 secciones.

1º) Preparar entorno
2º) Preparar fichero con hashes
3º) Ejecución de hashcat

In [ ]:
!echo -e "\n\e[1;34mPreparación del entorno\e[0m (tardara un par de minutos mínimo). Esta parte sólo hace falta ejecutarla una vez."               
!echo "Buscando paquetes nuevos..."
!apt update -y -q > /dev/null 2>&1
!echo "Actualizando paquetes..."
!apt upgrade -y -q > /dev/null 2>&1
!echo "Instalando hascat ..."
!apt install hashcat -y -q > /dev/null 2>&1
!echo "Instalando utilidades de pci ..."
!apt install pciutils -y -q > /dev/null 2>&1
!echo "Comprobando tarjeta gráfica..."
GPU_CARD = !lspci | grep NVIDIA
if not GPU_CARD:
    !echo -e "\n\e[31;1m[!] No se ha encontrado una GPU NVIDA! (Revisar: Editar->Configuración del cuaderno->Aceleración por Hardware->GPU\e[34;1m"
    !lspci
else:
    !echo -e "\e[1;34mEncontrada tarjeta gráfica NVIDIA\e[0m"
    !echo -e "\e[1;34mTest de Velocidad\e[0m" 
    !hashcat --benchmark -m1000 > TestDeVelocidad.txt 2>/dev/null
    !cat TestDeVelocidad.txt|grep Device
    !cat TestDeVelocidad.txt|grep Speed
    !echo -e "\e[1;34mListo para trabajar.\e[0m"

In [ ]:
!rm ficheroHASHES.txt 2>/dev/null
tipoHASH=1000   #tipo NTLM   #https://www.browserling.com/tools/ntlm-hash

#!echo _ >>ficheroHASHES.txt # _ en NTLM
#!echo _ >>ficheroHASHES.txt # _ en NTLM
!echo 93CC2B4D5D1FCCC38FAB249A9AEDB1E3 >>ficheroHASHES.txt # c en NTLM
!echo 8D3647093E662B9CF9D19C804C4655B6 >>ficheroHASHES.txt # ? en NTLM
!echo BF2A48EF8AC2E1705E93E823D9D2F7D7 >>ficheroHASHES.txt # co en NTLM
!echo 59C457395A6179A3784FB6DDA1CB9D14 >>ficheroHASHES.txt # c# en NTLM
!echo 00557E9D1DCD89164042625EF5DBA40C >>ficheroHASHES.txt # cos en NTLM
!echo 4613A1B81BF538B494FAB43CB9108EA1 >>ficheroHASHES.txt # co% en NTLM
!echo 4BF850B7E390F2F437603942423187C4 >>ficheroHASHES.txt #cosa en NTLM
!echo 452C129E4FB354AD1D7D44F52236C2CB >>ficheroHASHES.txt #cos¿ en NTLM
!echo B57F325A2942488FD508C1C130D13682 >>ficheroHASHES.txt #cosuca en NTLM
!echo F700E1777EAD870B6543CEB06583E074 >>ficheroHASHES.txt #usuaria en NTLM   
!echo F2AB082FA1B21C772EEA4193D454D7B0 >>ficheroHASHES.txt #usuario en NTLM   
!echo 2E910C3936F4836AC29BBDC307536524 >>ficheroHASHES.txt #usuario@1 en NTLM
!echo $HASH >>ficheroHASHES.txt

In [ ]:
#!rm ./potfile.txt          Los resultados intermedios se guardan en el fichero potfile.txt (se puede borrar)
for mascara in ['?a?a?a?a?a?a?a?a?a?a?a?a?a','?l?l?l?l?l?l?l?s?d',]:   #otros posibles intentos '?l?l?l?l?l?l?l?s?d','?u?l?l?l?l?l?l?s?d', '?u?u?u?u?u?u?u?u?u', '?a?a?a?a?a?a?a?a?a'
    !echo -e "\e[0;34mBuscando \e[1;34m$mascara\e[0m"
    !hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt --increment  ficheroHASHES.txt $mascara #> /dev/null 2>&1
    SinEncontrar=!hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt --increment --quiet ficheroHASHES.txt $mascara --left
    !echo -e "\n\n\n\e[1;34m-----------------------------------------------------------------------------------------------\e[0m"
    !echo -e "\e[1;34mResueltos hasta ahora!\e[0;34m"
    !hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt --increment --quiet ficheroHASHES.txt $mascara --show
    !echo -e "\e[1;34m-----------------------------------------------------------------------------------------------\e[0m"
    if not SinEncontrar:
        !echo -e "\n\e[1;34mResueltos todos los hashes!\e[0m"
        break


In [ ]:
#Trabajando como hilos
import threading
import time, datetime
def ContenidoPotFile( ficheroPot="./potfile.txt")
    !echo -e "\n\e[1;34m-----------------------------------------------------------------------------------------------\e[0m"
    !cat ./potfile.txt #!hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt --increment --quiet ficheroHASHES.txt $mascaraausar --show
    !echo -e "\e[1;34m-----------------------------------------------------------------------------------------------\e[0m"


def EjecutaHashcatDeMascara(mascaraausar, tipoHASH=1000, tipoDeIncremento="--increment", ficheroPot="./potfile.txt"):
    inicio=datetime.datetime.now()
    !echo -e "\e[0;34mBuscando \e[1;34m"$mascaraausar"\e[0m"
    !hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt --increment  ficheroHASHES.txt $mascaraausar > /dev/null 2>&1
    SinEncontrar=!hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt --increment --quiet ficheroHASHES.txt $mascaraausar --left
    fin=datetime.datetime.now()
    segundosEmpleados=round((fin-inicio).total_seconds(),2)
    !echo -e "\e[1;34mFin de máscara " $mascaraausar ". Resueltos ("$segundosEmpleados"sg):\e[0;34m"
    ContenidoPotFile(ficheroPot)

!rm ./potfile.txt
for mascara in ['?l?l?l?l?l?l?l?l?l?l?l?l']: #, "?a?l?a?a"
  x = threading.Thread(target=EjecutaHashcatDeMascara, args=(mascara,), daemon=False)
  x.start()
  contadorsegundos=0
  while x.isAlive():
      time.sleep(60*5)
      print('Esperando ' , contadorsegundos/60 , 'mn....')
      ContenidoPotFile
      contadorsegundos=contadorsegundos+1


In [ ]:
!hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt --increment --increment-min 8 ficheroHASHES.txt '?u?l?l?l?a?a?a?a?a?a?a?a?a' #> /dev/null 2>&1

In [ ]:
#Para ver la ayuda de hascat completa:
!hashcat --help